In [1]:
import numpy as np
import pandas as pd
import zipfile

In [2]:
# porosity and water saturation
# leased 360 square miles = 230400 acres
# thickness is 50 feet with standard deviation of 29.7
# formation volume factor = 1.6
def ooip(porosity, saturation, area = 230400, thickness = 50, fvf = 1.6):
    return 7758 * area * thickness * porosity * (1-saturation) / fvf

In [3]:
"""
returns a list containing deliverables in the followng order:
    A. well name
    B. easting
    C. northing
    D. length of well
    E. number of frac stages
    F. amount of proppant for each stage
    G. pump rate
    H. original oil in place
    I. recoverable reserves
    J. estimated ultimate recovery
"""
def deliverables(well_name, easting, northing):
    deliver = []
    
    # ABC: well name, easting, northing
    deliver.append(well_name)
    deliver.append(easting)
    deliver.append(northing)
    
    # data is the csv based on well_name
    zf = zipfile.ZipFile(r'well_production.zip')
    data = pd.read_csv(zf.open(well_name+'.csv'))
    
    # D: length
    deliver.append(data['easting'].max() - data['easting'].min())
    
    proppant_points = data.notna()  # This process finds the number of non-NaN rows
    proppant_data = data[proppant_points.all(axis=1)]
    
    # E: frac stages
    frac = len(proppant_data)
    deliver.append(frac)
    
    # F: average proppant for each stage
    deliver.append(proppant_data['proppant weight (lbs)'].sum() / frac)
    
    # G: average pump rate
    deliver.append(proppant_data['pump rate (cubic feet/min)'].sum() / frac)
    
    # H: original oil in place
    east_col = data['easting']
    index = east_col[east_col == easting].index[0]
    
    porosity = data['porosity'][index]
    saturation = data['water saturation'][index]
    deliver.append(ooip(porosity, saturation))
    
    # I: recoverable reserves
    deliver.append(-1)
    
    # J: estimated ultimate recovery
    deliver.append(-1)
    
    return deliver

In [4]:
# placeholder top 10 well names, eastings, northings
well_deliverables = []
names = ['Tribal 14-11H', 'Tribal 14-11H', 'Tribal 14-11H', 'Tribal 14-11H', 'Tribal 14-11H', 'Tribal 14-11H', 'Tribal 14-11H', 'Tribal 14-11H', 'Tribal 14-11H', 'Tribal 14-11H',]
eastings = [33447, 33447, 33447, 33447, 33447, 33447, 33447, 33447, 33447, 33447,]
northings = [80500, 80500, 80500, 80500, 80500, 80500, 80500, 80500, 80500, 80500,]

for i in range(10):
    well_deliverables.append(deliverables(names[i], eastings[i], northings[i]))

In [5]:
# creating DataFrame from list top 10 wells
col_names = ['well name', 'easting (ft)', 'northing (ft)', 
             'length of well (ft)', 'frac stages', 'average proppant per frac stage (lb)', 
             'average pump rate (cubic feet/min)', 'original oil in place (bbl)',
            'recoverable reserves (bbl)', 'estimated ultimate recovery (bbl)']

all_deliverables = pd.DataFrame(well_deliverables, columns = col_names)
all_deliverables

,well name,easting (ft),northing (ft),length of well (ft),frac stages,average proppant per frac stage (lb),average pump rate (cubic feet/min),original oil in place (bbl),recoverable reserves (bbl),estimated ultimate recovery (bbl)
0,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
1,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
2,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
3,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
4,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
5,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
6,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
7,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
8,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1
9,Tribal 14-11H,33447,80500,4898.0,50,871618.753545,300.345427,5.356579e+08,-1,-1


In [6]:
total_ooip = all_deliverables['original oil in place (bbl)'].sum()
total_rr = all_deliverables['recoverable reserves (bbl)'].sum()
total_eur = all_deliverables['estimated ultimate recovery (bbl)'].sum()

totals = pd.DataFrame([[total_ooip, total_rr, total_eur]], columns = ['total original oil in place (bbl)', 
                                                                      'total recoverable reserves (bbl)', 
                                                                      'total estimated ultimate recovery (bbl)'])
totals

,total original oil in place (bbl),total recoverable reserves (bbl),total estimated ultimate recovery (bbl)
0,5.356579e+09,-10,-10
